In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./1.menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests

                import sys
                sys.path.append('../00.Tools')
                from crawler_configuration import get_header, get_proxy

                resp = requests.get(input_.loc[a, 'Page Url'],
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(resp.strip(), 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_script = [text.strip() for text in html.xpath('//script[@data-product-compare-data]/text()')]

                # = = = = = = = = = = = = = = =

                import json

                list_json = [json.loads(script.strip()) for script in list_script]

                # = = = = = = = = = = = = = = =

                if len(list_json) == 0:
                    continue

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Page': input_.loc[a, 'Page'],
                                        'Page Url': input_.loc[a, 'Page Url'],
                                        'No': [i+1 for i in range(len(list_json))],
                                        'Handle': [f'''https://jaeautoelectrical.com.au/products/{json_['handle'].strip()}.js''' for json_ in list_json]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Page': input_.loc[a, 'Page'],
                                     'Page Url': input_.loc[a, 'Page Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'Page']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['Page', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct['No'] = [i+1 for i in range(len(output_correct))]
    output_correct.to_excel(f'''./2.url-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['Page'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./url_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：17

[状态：ok，尝试次数：  1] - 14
[剩余数量：0] - [当前时间：16:10:11]

[状态：ok，尝试次数：  1] - 17
[剩余数量：0] - [当前时间：16:10:11]

[状态：ok，尝试次数：  1] - 10
[剩余数量：0] - [当前时间：16:10:11]

[状态：ok，尝试次数：  1] - 7
[剩余数量：0] - [当前时间：16:10:12]

[状态：ok，尝试次数：  1] - 4
[剩余数量：0] - [当前时间：16:10:12]

[状态：ok，尝试次数：  1] - 11
[剩余数量：0] - [当前时间：16:10:12]

[状态：ok，尝试次数：  1] - 12
[剩余数量：0] - [当前时间：16:10:12]

[状态：ok，尝试次数：  1] - 13
[剩余数量：0] - [当前时间：16:10:13]

[状态：ok，尝试次数：  1] - 5
[剩余数量：0] - [当前时间：16:10:13]

[状态：ok，尝试次数：  1] - 16
[剩余数量：0] - [当前时间：16:10:13]

[状态：ok，尝试次数：  1] - 2
[剩余数量：0] - [当前时间：16:10:14]

[状态：ok，尝试次数：  1] - 15
[剩余数量：0] - [当前时间：16:10:14]

[状态：ok，尝试次数：  1] - 3
[剩余数量：0] - [当前时间：16:10:14]

[状态：ok，尝试次数：  1] - 6
[剩余数量：0] - [当前时间：16:10:15]

[状态：ok，尝试次数：  1] - 1
[剩余数量：0] - [当前时间：16:10:19]

[状态：ok，尝试次数：  1] - 9
[剩余数量：0] - [当前时间：16:10:20]

[状态：ok，尝试次数：  1] - 8
[剩余数量：0] - [当前时间：16:10:28]

输出ing...

Done ~
